# Inclusión de variables categóricas en los modelos

Por último analizaremos el impacto de incluir variables categóricas en nuestros modelos.

Esta es una pregunta bastante común en problemas de modelado. ¿Qué tanto cambia el resultado a través de diferentes categorías?
Las categorías son variables discretas sin un orden específico. Por ejemplo:

- Sexo: masculino, femenino.
- Continente: América, Asia, África.
- Universidad: ITESO, Tecnológico de Monterrey, UDG.

Hay que tener cuidado, ya que en muchos contextos, este tipo de variables se codifican como variables numéricas (one-hot encoding, label encoding, entre otros) y se suelen tratar en los modelos como variables continuas, lo cual puede llevar a conclusiones incorrectas.

Una idea un poco menos común es que los parámetros del modelo dependan de las categorías. En este caso, podríamos decir que desarrollamos un modelo por cada categoría. De esta forma, no se sufren de los problemas antes mencionados, y es lo que estudiaremos en este cuaderno.

Usaremos todo lo que vimos en el cuaderno anterior:

In [ ]:
# Importar pandas

# Importar pymc

# Importar arviz

# Importar numpy

# Importar train_test_split

# Importamos precision_recall_curve y f1_score

# Importamos pyplot

# Importamos seaborn


In [ ]:
# Leer el archivo de datos (bank.csv)


In [ ]:
# Versión numérica de la columna "deposit"


In [ ]:
# Normalización de las columnas numéricas


Utilizaremos solo una variable categórica correspondiente a:

housing: tiene préstamo hipotecario? (categórica: 'no', 'yes')

Para esto la codificamos primero:

In [ ]:
# Versión numérica de la columna poutcome


El modelo seguirá siendo básicamente el mismo, nada más que $\alpha$ dependerá de la variable poutcome.

$$
\begin{align}
\begin{array}{lcl}
deposit_i & \sim & \text{Bernoulli}(p_i) \\
\text{logit}(p_i) & = & \alpha_{POUTCOME[i]} + \beta_{balance}balance_i + \beta_{pdays}pdays_i + \beta_{previous}previous_i \\
\alpha_j & \sim & \text{Normal}(0, 1) \qquad \text{for } j=0,1,2,3\\
\beta_{balance} & \sim & \text{Normal}(0, 1) \\
\beta_{pdays} & \sim & \text{Normal}(0, 1) \\
\beta_{previous} & \sim & \text{Normal}(0, 1) \\
\end{array}
\end{align}
$$

In [ ]:
# Partición de los datos en entrenamiento y prueba


In [ ]:
# Redefinimos modelo con variables mutables


    # Variables numéricas mutables
    
    # Previas
    
    # Regresión
    
    # Deposit
    
    # Muestreo de la distribución posterior


In [ ]:
# az.plot_posterior


### Conclusión:

- Todas las variables siguen teniendo un impacto positivo en el objetivo. Sin embargo:
  - balance: tiene un impacto menor que cuando el modelo no consideraba housing.
  - pdays: tiene un impacto mayor que cuando el modelo no consideraba housing.
  - previous: tiene un impacto similar.

In [ ]:
# Importar scipy.special.expit as logistic


In [ ]:
# Plot posterior


### Conclusión:
- Observamos que la probabilidad de aceptar un depósito de un sujeto promedio cuando **no tiene crédito hipotecario**, es considerablemente mayor que cuando sí lo tiene.

In [ ]:
# Cambiamos los datos observados y muestreamos la distribución posterior predictiva


In [ ]:
# Tomamos promedio sobre las cadenas y las muestras


In [ ]:
# Construimos la curva precision-recall
precision, recall, thresholds = precision_recall_curve(y_true=y_test, y_score=y_score)
pr_curve = pd.DataFrame({'Precision': precision, 'Recall': recall})

In [ ]:
# Construimos la curva F1 vs. umbrales
f1 = pd.Series({t: f1_score(y_true=y_test, y_pred=y_score > t) for t in thresholds})
# Encontramos el umbral que maximiza F1
best_threshold = f1.idxmax()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(13, 5))

# Curva Precision-Recall, con el mejor umbral marcado
sns.scatterplot(x='Recall', y='Precision', data=pr_curve, ax=axes[0])
axes[0].plot(
    pr_curve.loc[np.where(thresholds == best_threshold)[0], 'Recall'],
    pr_curve.loc[np.where(thresholds == best_threshold)[0], 'Precision'],
    'ro',
    ms=10
)
axes[0].set_ylim(0,1)
axes[0].set_title('Precision-Recall Curve')

# Curva F1 vs. umbrales, con el mejor umbral marcado
f1.plot(ax=axes[1], title='F1 Scores', ylim=(0,1))
axes[1].set_xlabel('Threshold')
axes[1].axvline(best_threshold, lw=1, ls='--', color='k')
axes[1].text(x=.60, y=.95, s=f'Max F1 @ {best_threshold:.2f}')
fig.tight_layout()
plt.subplots_adjust(top=.8)
plt.show();

In [ ]:
# Accuracy


### Conclusión:

Incluir la variable categórica "housing" mejoró significativamente la capacidad predictiva de nuestro modelo.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>